In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
sns.set()

In [2]:
data = pd.read_csv('multiple_linear_regression.csv')
data.head()

,SAT,GPA,"Rand 1,2,3"
0,1714,2.40,1
1,1664,2.52,3
2,1760,2.54,3
3,1685,2.74,3
4,1693,2.83,2


In [3]:
data.describe()

,SAT,GPA,"Rand 1,2,3"
count,84.000000,84.000000,84.000000
mean,1845.273810,3.330238,2.059524
std,104.530661,0.271617,0.855192
min,1634.000000,2.400000,1.000000
25%,1772.000000,3.190000,1.000000
50%,1846.000000,3.380000,2.000000
75%,1934.000000,3.502500,3.000000
max,2050.000000,3.810000,3.000000


In [4]:
x = data[['SAT', 'Rand 1,2,3']]
y = data['GPA']
reg = LinearRegression()
reg.fit(x, y)

LinearRegression()

In [5]:
reg.coef_ #

array([ 0.00165354, -0.00826982])

In [6]:
reg.intercept_

np.float64(0.29603261264909575)

In [7]:
r_squared = reg.score(x, y) # r squared
r_squared

0.4066811952814282

# adjusted r squared 

$R^2_{adj.} = 1 - (1-R^2)*\frac{n-1}{n-p-1}$

In [8]:
x.shape # n is num of observations 84, p is num of predictors 2

(84, 2)

In [9]:
n = x.shape[0]
p = x.shape[1]
adjusted_r_squared = 1 - (1-r_squared) * (n-1) / (n-p-1)
adjusted_r_squared # it is lower, so our rand var have little or no explanatory power

0.39203134825134

In [10]:
# feature selection - simplify models
# if a var has p val above 0.05 we can disregard it 
# f regression creates lin.reg. of each feature and dependent var

In [11]:
from sklearn.feature_selection import f_regression
f_regression(x, y) # 2 arrays -> f-statistics and p-values
p_values = f_regression(x, y)[1]
p_values.round(3)

array([0.   , 0.676])

In [12]:
reg_summary = pd.DataFrame(data = x.columns.values, columns = ['Features'])
reg_summary['Coefficients'] = reg.coef_
reg_summary['p_values'] = p_values.round(3)
reg_summary

,Features,Coefficients,p_values
0,SAT,0.001654,0.000
1,"Rand 1,2,3",-0.008270,0.676


In [13]:
# standard scaler 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler # subtract mean and divide by stdev

StandardScaler()

In [14]:
scaler.fit(x)
x_scaled = scaler.transform(x)
x_scaled[:3]

array([[-1.26338288, -1.24637147],
       [-1.74458431,  1.10632974],
       [-0.82067757,  1.10632974]])

In [15]:
reg = LinearRegression()
reg.fit(x_scaled, y)

LinearRegression()

In [16]:
reg.coef_

array([ 0.17181389, -0.00703007])

In [17]:
reg.intercept_

np.float64(3.330238095238095)

In [18]:
reg_summary = pd.DataFrame([['Bias'], ['SAT'], ['Rand 1,2,3']], columns = ['Features'])
reg_summary['Weights'] = reg.intercept_, reg.coef_[0], reg.coef_[1]
reg_summary
# weights mean coefficients
# intercept is bias

,Features,Weights
0,Bias,3.330238
1,SAT,0.171814
2,"Rand 1,2,3",-0.007030


In [19]:
new_data = pd.DataFrame(data=[[1700, 2], [1800, 1]], columns=['SAT', 'Rand 1,2,3'])
#reg.predict(new_data)
new_data_scaled = scaler.transform(new_data)
reg.predict(new_data_scaled)

array([3.09051403, 3.26413803])

In [20]:
reg_simple = LinearRegression()
x_simple_matrix = x_scaled[:,0].reshape(-1, 1) # without rand var
reg_simple.fit(x_simple_matrix, y)
reg_simple.predict(new_data_scaled[:,0].reshape(-1, 1))

array([3.08970998, 3.25527879])